Vamos começar com a questão das derivadas, principalmente de forma numérica. Normalmente queremos
avaliar uma derivada em um ponto especifico. Caso tenhamos a função, é facil resolver
analiticamente, porem, quando temos uma tabela de dados, por exemplo, fica um pouco mais complexo.
Para resolver, poderiamos talvez fazer uma interpolação ou uma curva de regressão, e aplicar a
diferenciação para essa nova função, porem, nem so por que ela aproxima bem uma função, significa
que a derivada vai ser bem aproximada. Outra forma é utilizar o método das diferenças finitas, ou
seja, diferenciação numerica. Em suma, ela faz uso da serie de Taylor, no ponto da função onde
estamos interessados em sabe a derivada.

Diferenciação analitica. Vamos nos lembrar da definição formal de derivadas
$$
\frac{\mathrm{d}f(x)}{\mathrm{d}x}|_{\tilde{x}} = \lim_{h \to 0} = \frac{f(\tilde{x}+h)-f(\tilde{x})}{h}
$$
Vamos nos utilizar da expansão de Taylor ao redor de $\tilde{x}$, onde vamos simplificar para apenas $x$
$$
f(x+h) = f(x) + hf^{\prime}(x) + \frac{h^2}{2} f^{\prime \prime}(x) + \frac{h^3}{4} f^{\prime \prime \prime}(x) + ...
$$
Podemos rearranjar facilmente como
$$
f^{\prime}(x) = \frac{f(x+h)-f(x)}{h} + O(h)
$$
Onde tal formula é chamada de primeira aproximação da diferença positiva. Ela é igual à que chegamos analiticamente anteriormente, salvo um erro $O(h)$, que, em tese, não é tão grande, caso $h$ seja pequeno o suficiente.

Similarmente, podemos fazer a mesma analise para uma diferença negativa, onde o desenvolvimento é o mesmo, apenas sendo para $f(x-h)$, nos dando a formula
$$
f^{\prime}(x) = \frac{f(x)-f(x-h)}{h} + O(h)
$$

Caso queiramos tentar melhorar nossa aproximação, podemos fazer uma expansão para $\frac{h}{2}$, de forma:
$$
f(x+\frac{h}{2}) = f(x) + \frac{h}{2}f^{\prime}(x) + \frac{h^2}{8} f^{\prime \prime}(x) + \frac{h^3}{48} f^{\prime \prime \prime}(x) + ...
$$
Onde nos da a formula
$$
f^{\prime}(x) = \frac{f(x+\frac{h}{2})-f(x-\frac{h}{2})}{h} + O(h^2)
$$
Que nos da a a primeira diferença central positiva.
Vamos implementar

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [5]:
def f(x):
    return np.exp(np.sin(2*x))
def fprime(x):
    return 2*np.exp(np.sin(2*x))*np.cos(2*x)
def calc_fd(f, x, h):
    """ Calculudo da primeira diferença positiva
        f: função que queremos aproximar
        x: valor de x que queremos saber
        h: passo"""
    fd = (f(x+h) - f(x))/h
    return fd
def calc_cd(f,x, h):
    """ Calculudo da primeira diferença central positiva
        f: função que queremos aproximar
        x: valor de x que queremos saber
        h: passo"""
    cd = (f(x+h/2) - f(x-h/2))/h
    return cd
x = 0.5
an = fprime(x)
hs = [10**(-i) for i in range(1,12)]
fds = [abs(calc_fd(f,x,h) - an) for h in hs]
cds = [abs(calc_cd(f,x,h) - an) for h in hs]

rowf = "{0:1e} {1:1.16f} {2:1.16f}"
print('h            abs. error in fd     abs. error in cd')
for h, fd, cd in zip(hs,fds,cds):
    print(rowf.format(h,fd,cd))

h            abs. error in fd     abs. error in cd
1.000000e-01 0.3077044583376249 0.0134656094697734
1.000000e-02 0.0260359156900742 0.0001350472493096
1.000000e-03 0.0025550421497806 0.0000013505120728
1.000000e-04 0.0002550180941236 0.0000000135077878
1.000000e-05 0.0000254969542519 0.0000000001495843
1.000000e-06 0.0000025492660578 0.0000000002500959
1.000000e-07 0.0000002564334673 0.0000000011382744
1.000000e-08 0.0000000189018428 0.0000000189018428
1.000000e-09 0.0000003741732106 0.0000000699159992
1.000000e-10 0.0000021505300500 0.0000021505300500
1.000000e-11 0.0000332367747395 0.0000111721462455


Poderiamos tambem definir outros diferenciações, como por exemplo, a segunda diferença positiva
$$
f^{\prime}(x) = \frac{4f(x+\frac{h}{2})-f(x-h)-3f(x)}{h} + O(h^2)
$$
Ou a segunda diferença central
$$
f^{\prime}(x) = \frac{27f(x+\frac{h}{2})+f(x-\frac{3}{2}h)-27f(x-\frac{h}{2})-f(x+\frac{3}{2}h)}{24h} + O(h^4)
$$
Essa segunda sendo a melhor, com um erro $O(h^4)$, mas quanto mais formos continuando, mais termos precisaremos calcular, oq eu chega em um ponto que ja não compensa mais.

Podemos aproximar a segunda derivada tambem, por meio de uma expansão de taylor similar
$$
f^{\prime \prime}(x) = 4\frac{f(x+\frac{h}{2})-f(x-\frac{h}{2})-2f(x)}{h^2} + O(h^2)
$$
A primeira diferença central da segunda derivada

O grande problema é que sempre supomos que teriamos essa função a nosso dispor, mas quase nunca isso é verdade, de forma que muitas vezes so vamos ter pontos em uma tabela, então vamos precisar por nos mesmos definir $h$ com o que tivermos em mão. Muitas vezes nossos pontos vao estar em uma grid, variando de um valor $a \to b$, de formula
$$
x_i = a + ih\\
i=0,1,2,...,n-1\\
\therefore\\
h = \frac{b-a}{n-1}
$$
Com essa definição, se tivermos 101 pontos de 0 a 5 e quisermos avaliar, por meio da diferença central, vamos precisar dobrar nosso valor de $h$ para podermos usar na nossa formula, portanto, podemos modificar nossa formula para
$$
f^{\prime}(x) = \frac{f(x+h)-f(x-h)}{2h} + O(h^2)
$$
Similarmente para segunda derivada
$$
f^{\prime \prime}(x) = \frac{f(x+h)-f(x-h)-2f(x)}{h^2} + O(h^2)
$$


Uma formula para extrapolação de valores pode ser derivada, caso esteja interessado ver a extrapolação de Richardson, onde nosso valor pode ser dado por

$$
G = \frac{2^p g(h/2) - g(h)}{2^p - 1} + O(h^{p+q})
$$
Vamos implementar

In [16]:
print('h          abs. err rich fd        abs. err. rich cd')
for h in hs:
    fdrich = 2*calc_fd(f, x, h/2) - calc_fd(f, x, h)
    fd = abs(fdrich- an)
    cdrich = (4*calc_cd(f, x, h/2)- calc_cd(f, x, h))/3
    cd = abs(cdrich - an)
    print(rowf.format(h, fd, cd))

h          abs. err rich fd        abs. err. rich cd
1.000000e-01 0.0259686059827384 0.0000098728370874
1.000000e-02 0.0002695720500006 0.0000000009897421
1.000000e-03 0.0000027005434182 0.0000000000011098
1.000000e-04 0.0000000270109117 0.0000000000043667
1.000000e-05 0.0000000003389138 0.0000000000280513
1.000000e-06 0.0000000006941852 0.0000000002500959
1.000000e-07 0.0000000100200586 0.0000000011382744
1.000000e-08 0.0000000189018428 0.0000000995219467
1.000000e-09 0.0000003741732106 0.0000005222029471
1.000000e-10 0.0000110323142470 0.0000080717195146
1.000000e-11 0.0000332367747395 0.0000703840408920


Existe tambem o método da auto-diferenciação, a qual chamamos nosso numero de 
$$
\mathbf{a} = a + a^{\prime}d
$$
Onde d so serve para indicar a existencia da componente $a^{\prime}$, onde definimos $d^2=0$. Com isso, todas suas operações fazem com que a parte sem $d$ sejam iguais às de um numero real e as com $d$ sigam as regras de diferenciação.

Vamos fazer agora um projeto de Mecanica quantica, para aplicarmos o que aprendemos sobre diferenciacao. A derivada faz-se de extrema importancia quanto ao ponto de calculo de energia cinetica na mecanica quantica. Para uma unica particula, sua equacao de oscilacao harmonica

$$
-\frac{\hbar^2}{2m} \frac{\mathrm{d}^2 \psi(x)}{\mathrm{d}x^2} + \frac{1}{2}m \omega^2 x^2 \psi (x) = E \psi (x)
$$
Onde $\hbar$ da-se como a constante de planck dividida por $2 \pi$ e $m$ se da como a massa da particula. Essa equacao envolve a energia cinetica, com o termo da segunda derivada e a energia potencial com o $x^2$ e o $\omega$ se da como a a frequencia angular classica.

Qualquer livro de mecanica quantica vai nos ensinar a resolver achando os autovalores da energia
$$
E_n = \left( n + \frac{1}{2} \right) \hbar \omega
$$
E os autovetores normalizados
$$
\psi_n(x) = \frac{1}{\sqrt{2^n n!}} \left(\frac{m \omega}{\pi \hbar}\right)^{1/4} H_n \left(\sqrt{\frac{m \omega}{\hbar}x}\right) e^{-m \omega x^2/(2 \hbar)}
$$
Vemos que nosso termo possui um termo gaussiano, que se assemeha a uma normal em juncao com um termo polinomial $H_n(x)$ que é conhecido como o Polinomio de Hermite, que obedece uma relacao de recorrencia similar ao polonio de Legendre, dado por
$$
H_{j+1}(x)= 2xH_j(x) - 2jH_{j-1}(x)
$$
Para comecarmos, precisamos definir os termos $H_0(x) = 1$ e $H_1(x) = 2x$. A derivada é definida como 
$$
H_n ^{\prime} = 2n H_{n-1}(x)
$$
Mas por hora nao vamos precisar delas.

Por  meio do principio da correspondencia, se tivermos um numero quantico muito alto, vamos ter uma volta a principior classicos. Vamos nos lembrar da energia total $E$, da mecanica classica, dada por
$$
E = \frac{1}{2}m \omega ^2 x_0^2
$$
Esse caso considera a energia cinetica como zero, tendo pontos criticos em $\pm x_0$. Onde nossa particula oscila entra $-x_0 \geq x \geq x_0$. Com isso, conseguimos derivar a nossa funcao densidade de probabilidade para a mecanica classica como
$$
P_c(x) = \frac{1}{\pi \sqrt{x_0^2 - x^2}}
$$
Se fizermos a media da probabilidade da mecanica quantica que é dada por 
$$
P(x) = |\psi (x)|^2
$$
quando $n \to \infty$ vamos ter a volta da mecanica classica.

Um problema classico de mecanica quantica é uma particula a qual esta oscilando em uma caixa, ou seja, esta preso em um local com uma parede de potencial infinito, o que significa que nossa equcao de onda vai para zero nas bordas. Se nossa caixa for de $-L/2 \to L/2$, nossa equcao de Schodinger dependente do tempo fica como
$$
- \frac{\hbar ^2}{2m} \frac{\mathrm{d}^2 \psi(x)}{\mathrm{d}x^2} = E \psi (x)
$$
Assumindo que nao temos potencial externo a caixa. As auto funcoes sao dadas por
$$
\phi_k (x) = \frac{1}{\sqrt{L}}e^{ikx}
$$
Por meio da nossa da condicao de contorno, podemos escreve-la $\phi_k = \phi_k (x+L)$, Por meio dessas duas equacoes podemos escrever como
$$
k = \frac{2 \pi}{L}n
$$
Substituindo essas relacoes na relacao de $E$ vamos ter
$$
E = \frac{\hbar ^2}{2m} \left(\frac{2 \pi}{L} \right)^2 n^2
$$
Vamos implementar

In [1]:
import numpy as np
import cmath
from math import exp

In [ ]:
def hermite(n,x):
     """
     n:numero de graus
     x: valor que queremos saber
     """
    val0 = 1
    val1 = 2*x
    for i in range(1,n):
        val2= 2*x*val1 - 2*i*val0
        val0 = val1
        val1 = val2
    dval = 2*n*val0
    return val2
def psiqho(x, nametoval):
    """
    nametoval: dicionario que contem
    n: numero quantico
    m_om_hbar: constante m omega planck
    alpha: parametro extra para a gaussiana
    """
    n = nametoval["n"]
    momhbar = nametoval["momhbar"]
    al = nametoval["alpha"]
    psival = momhbar**0.25 * np.exp(-0.5*al*momhbar * x**2)
    psival /= np.sqrt(2**n *  np.math.factorial(n) * np.sqrt(np.pi))
def psibox(x, nametoval):
    """
    nametoval: dicionario que contem
    n: numero quantico
    boxl: tamanho da caixa
    """
    n = nametoval["n"]
    boxl = nametoval["boxl"]
    return cmath.exp(2*np.pi*x*1j/boxl)
